## Dependencies

In [1]:
# Confirmed needed dependencies
import pandas as pd
import sys

# Dependencies for mapping
import gmplot

# Dependency for Heat Mapper
import gmaps


from config import google_API_Key

## File inputs/outputs

In [2]:
randLatLon_csv = "./Data/Archived/randomLatLon.csv" 
addressList_csv = "./Data/Archived/addressList.csv"
masterDataCLEAN_csv = "./Data/masterDataCLEAN.csv"


## Helpful Functions

In [ ]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig